In [1]:
%load_ext autoreload

%autoreload 2
from utils import parse_table, read_csv
import pandas as pd
import numpy as np

from pretty import plot_kde, ColorTheme


def get_scatter(in_distr, ood, type, all_tasks): 
    global likelihood_generated, lid_generated
    global likelihood_in, lid_in
    global likelihood_ood, lid_ood
    in_vs_out = all_tasks[type][in_distr][ood]
    likelihood_generated = in_vs_out[in_vs_out['name'] == 'generated']['log-likelihood'].values
    lid_generated = in_vs_out[in_vs_out['name'] == 'generated']['LID'].values
    likelihood_in = in_vs_out[in_vs_out['name'] == 'test']['log-likelihood'].values
    lid_in = in_vs_out[in_vs_out['name'] == 'test']['LID'].values
    likelihood_ood = in_vs_out[in_vs_out['name'] == 'ood']['log-likelihood'].values
    lid_ood = in_vs_out[in_vs_out['name'] == 'ood']['LID'].values

print_mapping = {
    'fashion-mnist': 'FMNIST',
    'mnist': 'MNIST',
    'emnist': 'EMNIST',
    'omniglot': 'Omniglot',
    'celeba-small': 'CelebA',
    'svhn': 'SVHN',
    'cifar10': 'CIFAR10',
    'cifar100': 'CIFAR100',
    'tiny-imagenet': 'Tiny',
}

df_grayscale = read_csv('grayscale_LID_vs_likelihood_diffusion_lpca.csv')
df_rgb = read_csv('rgb_LID_vs_likelihood_diffusion_lpca.csv')
all_grayscale_tasks = parse_table(df_grayscale)
all_rgb_tasks = parse_table(df_rgb)
all_tasks = {
    'grayscale': all_grayscale_tasks,
    'rgb': all_rgb_tasks,
}

all_pairs = []
for tp in all_tasks.keys():
    for in_distr in all_tasks[tp].keys():
            for ood in all_tasks[tp].keys():
                if in_distr != ood:
                    all_pairs.append((in_distr, ood, tp))

5/7: 100%|██████████| 5/5 [00:29<00:00,  5.97s/it]


In [5]:
from roc_analysis import get_auc, get_convex_hull, get_roc_graph

COMPUTE_LIMIT = 100000

for in_distr, ood, tp in all_pairs:
    print(f'{print_mapping[in_distr]} vs {print_mapping[ood]}')
    get_scatter(in_distr, ood, tp, all_tasks)
    x_naive, y_naive = get_roc_graph(
        pos_x = likelihood_in,
        neg_x = likelihood_ood,
        verbose=0,
    )
    x_curve, y_curve = get_convex_hull(x_naive, y_naive)
    likelihood_test_vs_ood_auc = get_auc(x_curve, y_curve)
    print("Likelihood AUC of test-vs-out", "{:.3f}".format(likelihood_test_vs_ood_auc))
    
    get_scatter(in_distr, ood, tp, all_tasks)
    x_naive, y_naive = get_roc_graph(
        pos_x = likelihood_generated,
        neg_x = likelihood_ood,
        verbose=0,
    )
    x_curve, y_curve = get_convex_hull(x_naive, y_naive)
    likelihood_generated_vs_ood_auc = get_auc(x_curve, y_curve)
    print("Likelihood AUC of generated-vs-out", "{:.3f}".format(likelihood_generated_vs_ood_auc))
    
    x_ours, y_ours = get_roc_graph(
        pos_x = likelihood_in,
        pos_y = lid_in,
        neg_x = likelihood_ood,
        neg_y = lid_ood,
        compute_limit=COMPUTE_LIMIT,
        verbose=0,
    )
    x_curve, y_curve = get_convex_hull(x_ours, y_ours)
    dual_test_vs_ood_auc = max(likelihood_test_vs_ood_auc, get_auc(x_curve, y_curve))
    print("dual threshold AUC of test-vs-out", "{:.3f}".format(dual_test_vs_ood_auc))
    
    x_ours, y_ours = get_roc_graph(
        pos_x = likelihood_generated,
        pos_y = lid_generated,
        neg_x = likelihood_ood,
        neg_y = lid_ood,
        compute_limit=COMPUTE_LIMIT,
        verbose=0,
    )
    x_curve, y_curve = get_convex_hull(x_ours, y_ours)
    dual_generated_vs_ood_auc = max(likelihood_generated_vs_ood_auc, get_auc(x_curve, y_curve))
    print("dual threshold AUC of generated-vs-out", "{:.3f}".format(dual_generated_vs_ood_auc))
    print("----")

Omniglot vs MNIST
Likelihood AUC of test-vs-out 0.995
Likelihood AUC of generated-vs-out 0.000
dual threshold AUC of test-vs-out 0.995
dual threshold AUC of generated-vs-out 0.558
----
Omniglot vs FMNIST
Likelihood AUC of test-vs-out 1.000
Likelihood AUC of generated-vs-out 0.000
dual threshold AUC of test-vs-out 1.000
dual threshold AUC of generated-vs-out 0.001
----
Omniglot vs EMNIST
Likelihood AUC of test-vs-out 1.000
Likelihood AUC of generated-vs-out 0.000
dual threshold AUC of test-vs-out 1.000
dual threshold AUC of generated-vs-out 0.082
----
MNIST vs Omniglot
Likelihood AUC of test-vs-out 1.000
Likelihood AUC of generated-vs-out 0.000
dual threshold AUC of test-vs-out 1.000
dual threshold AUC of generated-vs-out 0.888
----
MNIST vs FMNIST
Likelihood AUC of test-vs-out 0.996
Likelihood AUC of generated-vs-out 0.000
dual threshold AUC of test-vs-out 0.996
dual threshold AUC of generated-vs-out 0.561
----
MNIST vs EMNIST
Likelihood AUC of test-vs-out 0.846
Likelihood AUC of gener